# PCA Example On Colors

Color is an example where using PCA *without* normalizing the data makes sense. Dimensions of color representation are generally comparable and on a similar scale, reducing the need for normalization to a particular data set.

Try this example with a few different images. It provides some a basis for comparing the relative merits of [RGB](https://en.wikipedia.org/wiki/RGB_color_model) vs. other representations such as [YCbCr](https://en.wikipedia.org/wiki/YCbCr) and [HSL](https://en.wikipedia.org/wiki/HSL_and_HSV).

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from skimage.io import imread # conda install scikit-image
from skimage.transform import rescale
import plotly.express as px

In [4]:
# Load an image

# Use your own JPEG file, or grab one such as this:
# https://edurant.smugmug.com/Portfolio/2007-Calendar/i-jFDZXNX/A

#filepath = 'IMG_5818.JPEG'
filepath = '06-IC7H2647c.jpg'
image = imread(filepath)
if True: # original size
    image = image.astype('float64')/256
else: # rescale to fixed area; speed up calculations for large images
    scale = 200/np.sqrt(np.prod(image.shape[:-1]))
    image = rescale(image, (scale,scale,1), anti_aliasing=True)
image.shape

FileNotFoundError: No such file: 'c:\Users\arneyh\CSC 5601 - Theory of Machine Learning\Other\In Class PCA\06-IC7H2647c.jpg'

In [ ]:
plt.imshow(image)
plt.title('Original Image')
plt.show()

# PCA on Original RGB Data

In [ ]:
data = image.reshape(-1, 3) # Flatten the image data for PCA on colors

pca = PCA(n_components=3)
transformed_data = pca.fit_transform(data)

print(data.dtype)
print(data.shape)
mean_pixel = np.mean(data, axis=0)
print("Mean pixel:", mean_pixel)
print("Pixel standard deviation:", np.std(data, axis=0))

In [ ]:
# Extract the principal components
components = pca.components_
print("Components:", components)
print("Explained Variance Ratio: ", pca.explained_variance_ratio_)

# Reconstruct the image using only the first principal component
reconstructed_1 = mean_pixel + np.outer(transformed_data[:, 0], components[0])
reconstructed_1_image = reconstructed_1.reshape(image.shape).clip(0, 1)

print("Image from 1 component…")
print("Mean squared error:", sklearn.metrics.mean_squared_error(image.flat, reconstructed_1_image.flat))
print("Shape:", reconstructed_1.shape)
print("Min:", np.min(reconstructed_1, axis=0))
print("Mean:", np.mean(reconstructed_1, axis=0))
print("Std:", np.std(reconstructed_1, axis=0))
print("Max:", np.max(reconstructed_1, axis=0))

# Reconstruct the image using the first two principal components
reconstructed_2 = reconstructed_1 + np.outer(transformed_data[:, 1], components[1])
reconstructed_2_image = reconstructed_2.reshape(image.shape).clip(0, 1)
print("2 PCs: Mean squared error:", sklearn.metrics.mean_squared_error(image.flat, reconstructed_2_image.flat))

# Reconstruct the image using the first two principal components
reconstructed_3 = reconstructed_2 + np.outer(transformed_data[:, 2], components[2])
reconstructed_3_image = reconstructed_3.reshape(image.shape).clip(0, 1)
print("3 PCs: Mean squared error:", sklearn.metrics.mean_squared_error(image.flat, reconstructed_3_image.flat))

# Display the reconstructed images
fig, ax = plt.subplots(1, 3, figsize=(12, 6))
ax[0].imshow(reconstructed_1_image)
ax[0].set_title('Reconstruction with 1st PC')
ax[1].imshow(reconstructed_2_image)
ax[1].set_title('Reconstruction with First 2 PCs')
ax[2].imshow(reconstructed_3_image)
ax[2].set_title('Reconstruction with First 3 PCs')
plt.show()

## Show the contribution of each color component

In [ ]:
weights = np.linspace(-2, 2, 400)

fig, axes = plt.subplots(1, 3, figsize=(10, 2))
for i in range(3):
    # Calculate the spectrum for each principal component
    spectrum = np.outer(weights, pca.components_[i]) + pca.mean_
    spectrum = np.clip(spectrum, 0, 1) # valid color range
    # Plot each spectrum
    axes[i].imshow([spectrum], aspect='auto')
    axes[i].set_title(f'Component {i+1}')
    axes[i].axis('off')
plt.tight_layout()
plt.show()

# Projection Where First Component is Brightness

In [ ]:
# Define the brightness vector
brightness = np.array([1, 1, 1]) / np.sqrt(3)
brightness_projection = data.dot(brightness)

# Remove the brightness component
data_without_brightness = data - np.outer(brightness_projection, brightness)

mean_pixel_b = np.mean(data_without_brightness, axis=0)
print("Brightness removed pixel data…")
print("Min:", np.min(data_without_brightness, axis=0))
print("Mean:", mean_pixel_b)
print("Mean of mean pixel:", np.mean(mean_pixel_b))
print("Std:", np.std(data_without_brightness, axis=0))
print("Max:", np.max(data_without_brightness, axis=0))

# PCA on the data from which brightness has been removed
pca_no_brightness = PCA(n_components=2)
transformed_no_brightness = pca_no_brightness.fit_transform(data_without_brightness)

# Reconstruct the image using only the brightness vector
reconstructed_1b = np.outer(brightness_projection, brightness)
reconstructed_1b_image = reconstructed_1b.reshape(image.shape).clip(0, 1)
print("Brightness: Mean squared error:", sklearn.metrics.mean_squared_error(image.flat, reconstructed_1b_image.flat))

# Add the projection of the first principal component after brightness removal
reconstructed_2b = mean_pixel_b + reconstructed_1b + np.outer(transformed_no_brightness[:, 0], pca_no_brightness.components_[0])
reconstructed_2b_image = reconstructed_2b.reshape(image.shape).clip(0, 1)
print("Brightness + x̄ + 1 PC: Mean squared error:", sklearn.metrics.mean_squared_error(image.flat, reconstructed_2b_image.flat))

# Add the projection of the second principal component after brightness removal
reconstructed_3b = reconstructed_2b + np.outer(transformed_no_brightness[:, 1], pca_no_brightness.components_[1])
reconstructed_3b_image = reconstructed_3b.reshape(image.shape).clip(0, 1)
print("Brightness + x̄ + 2 PCs: Mean squared error:", sklearn.metrics.mean_squared_error(image.flat, reconstructed_3b_image.flat))

# Display the reconstructed images
fig, ax = plt.subplots(1, 3, figsize=(12, 6))
ax[0].imshow(reconstructed_1b_image)
ax[0].set_title('Brightness')
ax[1].imshow(reconstructed_2b_image)
ax[1].set_title('Brightness + x̄ + 1 PCA Component')
ax[2].imshow(reconstructed_3b_image)
ax[2].set_title('Brightness + x̄ + 2 PCA Components')
plt.show()

## Show the contribution of each color component

In [ ]:
weights = np.linspace(-2, 2, 400)
fig, axes = plt.subplots(2, 2, figsize=(10, 2))
for i in range(2):
    # Calculate the spectrum for each principal component
    spectrum_cent = np.outer(weights, pca_no_brightness.components_[i])
    spectrum = spectrum_cent + mean_pixel_b
    # Plot each spectrum
    axes[0,i].imshow([spectrum.clip(0,1)], aspect='auto') # clip to valid color range
    axes[0,i].set_title(f'Component {i+1} (Centered & Shifted)')
    axes[0,i].axis('off')
    axes[1,i].imshow([spectrum_cent.clip(0,1)], aspect='auto')
    axes[1,i].axis('off')

plt.tight_layout()
plt.show()